In [1]:
from __future__ import print_function, division
from tensorflow import keras
import tensorflow as tf

from keras.layers import *
from keras.models import Model, Sequential
from keras.optimizers import Adam, RMSprop
import keras.backend as K

from math import log2
import numpy as np
from random import randint

import matplotlib.pyplot as plt
import os

In [2]:
# Удаляем все прошлые изображения
for i in os.listdir("./generated_flowers"):
    os.remove(f"./generated_flowers/{i}")

class CCGAN(keras.Model):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.NUM_CLASSES = 5  # Не менять!

        # Входные форматы
        self.IMG_SHAPE = (88, 88, 3)
        self.LATENT_DIM = 2
        self.HANDICAP = 0  # Фора чтобы одна иишка не отставала от другой

        # Константы
        self.FILTERS_DIS = 2048   # Нижняя граница
        self.FILTERS_GEN = 2048
        self.DROPOUT = 0.0
        self.HIDDEN_IMG_SHAPE = (80, 80, 1)

        # Чем меньше тем лучше:
        self.AMOUNT_DISCRIMINATOR_LAYERS = 3
        self.AMOUNT_GENERATOR_LAYERS = 3

        """
        Генератор и Дискриминатор
        """
        # Мучаемся со входами
        self.image_inp = Input(shape=self.IMG_SHAPE, name="image")
        self.label_inp = Input(shape=(self.NUM_CLASSES,), name="label")
        self.latent_space_inp = Input(shape=(self.LATENT_DIM,), name="latent_space")

        # Создаем дискриминатор
        self.build_discriminator()
        self.discriminator.summary()
        # Создаем генератор
        self.build_generator()
        self.generator.summary()

        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.decode_loss_tracker = keras.metrics.Mean(name="decode_loss")
        self.bias_loss_tracker = keras.metrics.Mean(name="bias_loss")

        """
        Модели
        """
        # z == latten_space
        self.generated_z = self.generator([self.latent_space_inp, self.label_inp])
        self.dis_gen_z = self.discriminator([self.generated_z, self.label_inp])

        ccgan_model = Model([self.latent_space_inp, self.label_inp], self.dis_gen_z, name="CCGAN")
        self.ccgan = ccgan_model([self.latent_space_inp, self.label_inp])

        self.optimizer_gen = Adam(1e-2)
        self.optimizer_dis = Adam(1e-2)

    def build_discriminator(self) -> Model:
        # # Объединяем картинку с лейблами
        # x = Embedding(self.NUM_CLASSES, self.IMG_SHAPE[0]**2)(self.label_inp)
        # x = Reshape([*self.IMG_SHAPE[:2], self.NUM_CLASSES])(x)
        # x = concatenate([self.image_inp, x])
        #
        # for i in range(1, self.AMOUNT_DISCRIMINATOR_LAYERS +1):
        #     x = AveragePooling2D()(x)
        #     x = Dropout(self.DROPOUT)(x)
        #     x = Conv2D(self.FILTERS_DIS * 2**i, (11, 11), activation="tanh")(x)
        #     x = BatchNormalization()(x)
        #
        # while x.shape[1] >= 3:
        #     x = Dropout(self.DROPOUT)(x)
        #     x = Conv2D(x.shape[-1], (3, 3), activation="tanh")(x)
        #
        # x = Flatten()(x)
        #
        # # Постепенно сжимаем
        # dense_units = x.shape[-1] // 2
        # while dense_units >= 4:
        #     x = Dropout(self.DROPOUT)(x)
        #     x = concatenate([self.label_inp, x])  # Добавляем метки класса
        #     x = Dense(dense_units, activation="tanh")(x)
        #     dense_units //= 2

        x = Flatten()(self.image_inp)
        x = Dense(self.FILTERS_DIS, activation="tanh")(x)

        while x.shape[-1] >= 2:
            x = Dropout(self.DROPOUT)(x)
            x = Dense(x.shape[-1] // 2, activation="tanh")(x)

        # Добавляем метки класса
        x = concatenate([self.label_inp, x])
        x = Dense(1, activation="sigmoid")(x)

        self.discriminator = Model([self.image_inp, self.label_inp], x, name="discriminator")

    def build_generator(self) -> Model:
        # # Разжимаем вектор шума в маленькую картинку
        # x = self.latent_space_inp
        #
        # for _ in range(self.AMOUNT_GENERATOR_LAYERS):
        #     x = Dropout(self.DROPOUT)(x)
        #     x = concatenate([x, self.label_inp])
        #     x = Dense(x.shape[-1]*2, activation="tanh")(x)
        #
        # x = concatenate([x, self.label_inp])
        # x = Dense(np.prod(self.HIDDEN_IMG_SHAPE), activation="tanh")(x)
        # x = Reshape(self.HIDDEN_IMG_SHAPE)(x)
        #
        # x = UpSampling2D()(x)
        # x = Conv2DTranspose(self.FILTERS_GEN, (11, 11), activation="tanh")(x)
        # x = BatchNormalization()(x)
        #
        # generated_img = Conv2D(3, (1, 1), activation="tanh")(x)

        x = self.latent_space_inp
        while x.shape[-1] < self.FILTERS_GEN:
            x = Dropout(self.DROPOUT)(x)
            x = Dense(x.shape[-1] * 2, activation="tanh")(x)

        x = Dense(np.prod(self.IMG_SHAPE), activation="tanh")(x)
        generated_img = Reshape(self.IMG_SHAPE)(x)

        self.generator = Model([self.latent_space_inp, self.label_inp], generated_img, name="generator")

    def batch_gen(self, batch_size, dataset):
        """Чтобы использовать "big_flowers_dataset" (расширенный датасет) надо запустить increasing_data.py"""
        train_data = keras.preprocessing.image_dataset_from_directory(
            dataset,
            image_size=self.IMG_SHAPE[:-1],
            label_mode="categorical",
            shuffle=True,
            batch_size=batch_size,
        )

        while True:
            # Добавляем лейблы (т.к. у нас CCGAN) и нормализуем в [-1; 1], т.к. юзаем tanh
            # (т.к. с sigmoid градиент затухает)
            x, y = next(iter(train_data))
            x = x / 127.5 -1
            noise = np.random.normal(0, 1, (batch_size, self.LATENT_DIM))

            yield x, y, noise

    def sample_images(self, epoch):
        row, column = 2, self.NUM_CLASSES
        noise = np.random.normal(0, 1, (row * column, self.LATENT_DIM))
        label = np.array([
            np.arange(0, self.NUM_CLASSES) for _ in range(row)
        ]).reshape((-1, 1))
        sampled_labels = keras.utils.to_categorical(label, self.NUM_CLASSES)

        gen_imgs = self.generator.predict([noise, sampled_labels], verbose=False)
        gen_imgs = np.array(gen_imgs)
        gen_imgs -= np.min(gen_imgs)
        gen_imgs /= np.max(gen_imgs)

        # Делаем картинку
        fig, axs = plt.subplots(row, column, figsize=(12, 6))
        count = 0
        for i in range(row):
            for j in range(column):
                axs[i, j].imshow(gen_imgs[count, :, :, :])
                axs[i, j].set_title(label[count][0])
                axs[i, j].axis("off")
                count += 1
        fig.savefig("generated_flowers/%d.png" % epoch)
        plt.close()

    def train(self, batch_size=32, dataset="flowers_dataset"):
        # Просто единицы и нули для Дискриминатора
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        get_batch = self.batch_gen(batch_size=batch_size, dataset=dataset)

        epoch_count = 1
        all_l_dis = [0]
        all_l_gen = [0]

        for learn_iter in range(int(10**10)):
            # Обучение дискриминатора
            for _ in range(1 + (self.HANDICAP if np.mean(all_l_dis)-0.15 > np.mean(all_l_gen) else 0)):
                images, labels, noise = next(get_batch)
                with tf.GradientTape() as dis_tape:
                    dis_real_output = self.discriminator([images, labels], training=True)
                    generated_images = self.generator([noise, labels], training=False)
                    dis_fake_output = self.discriminator([generated_images, labels], training=True)

                    # Чем настоящие картинки нереальнее или сгенерированные реальные, тем ошибка больше
                    l_dis = 0.5 * (tf.reduce_mean(-tf.math.log(dis_real_output + 1e-9)) +
                                   tf.reduce_mean(-tf.math.log(1. - dis_fake_output + 1e-9)))

                all_l_dis.append(l_dis)

                # Получаем градиенты для дискриминатора
                grads_dis = dis_tape.gradient(l_dis, self.discriminator.trainable_variables)
                self.optimizer_dis.apply_gradients(zip(grads_dis, self.discriminator.trainable_variables))

            # Обучение генератора
            for _ in range(1 + (self.HANDICAP if np.mean(all_l_gen)-0.15 > np.mean(all_l_dis) else 0)):
                images, labels, noise = next(get_batch)
                with tf.GradientTape() as gen_tape:
                    generated_images = self.generator([noise, labels], training=True)
                    dis_output = self.discriminator([generated_images, labels], training=False)

                    # Чем более реалистичная картина (для дискриминатора), тем меньше ошибка
                    l_gen = -tf.reduce_mean(tf.math.log(dis_output + 1e-9))

                all_l_gen.append(l_gen)

                # Получаем градиенты для генератора
                grads_gen = gen_tape.gradient(l_gen, self.generator.trainable_variables)
                self.optimizer_gen.apply_gradients(zip(grads_gen, self.generator.trainable_variables))

            # Сохраняем генерируемые образцы каждую эпоху
            if learn_iter % (2800 // batch_size) == 0:
                self.sample_images(epoch_count)

                # Вывод прогресса и средних ошибок
                print(f"{epoch_count:02} \t"
                      f"[Dis loss: {np.mean(all_l_dis):.3f}] \t"
                      f"[Gen loss: {np.mean(all_l_gen):.3f}]")

                epoch_count += 1
                all_l_dis = [0]
                all_l_gen = [0]


ccgan = CCGAN()
print("Generator:    ", f"{ccgan.generator.count_params():,}")
print("Discriminator:", f"{ccgan.discriminator.count_params():,}")
print("Sum:          ", f"{ccgan.generator.count_params() + ccgan.discriminator.count_params():,}")

Model: "discriminator"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 image (InputLayer)             [(None, 88, 88, 3)]  0           []                               
                                                                                                  
 flatten (Flatten)              (None, 23232)        0           ['image[0][0]']                  
                                                                                                  
 dense (Dense)                  (None, 2048)         47581184    ['flatten[0][0]']                
                                                                                                  
 dropout (Dropout)              (None, 2048)         0           ['dense[0][0]']                  
                                                                                      

In [3]:

ccgan.train(batch_size=256, dataset="flowers_dataset")


Found 2799 files belonging to 5 classes.
01 	[Dis loss: 0.368] 	[Gen loss: 0.287]
02 	[Dis loss: 0.626] 	[Gen loss: 0.542]
03 	[Dis loss: 0.588] 	[Gen loss: 0.660]
04 	[Dis loss: 0.543] 	[Gen loss: 0.777]
05 	[Dis loss: 0.536] 	[Gen loss: 0.825]
06 	[Dis loss: 0.589] 	[Gen loss: 0.720]
07 	[Dis loss: 0.629] 	[Gen loss: 0.656]
08 	[Dis loss: 0.625] 	[Gen loss: 0.759]
09 	[Dis loss: 0.621] 	[Gen loss: 0.766]
10 	[Dis loss: 0.624] 	[Gen loss: 0.744]
11 	[Dis loss: 0.619] 	[Gen loss: 0.716]
12 	[Dis loss: 0.616] 	[Gen loss: 0.703]
13 	[Dis loss: 0.612] 	[Gen loss: 0.694]
14 	[Dis loss: 0.621] 	[Gen loss: 0.693]
15 	[Dis loss: 0.618] 	[Gen loss: 0.691]
16 	[Dis loss: 0.615] 	[Gen loss: 0.693]
17 	[Dis loss: 0.618] 	[Gen loss: 0.687]
18 	[Dis loss: 0.615] 	[Gen loss: 0.681]
19 	[Dis loss: 0.617] 	[Gen loss: 0.695]
20 	[Dis loss: 0.611] 	[Gen loss: 0.691]
21 	[Dis loss: 0.617] 	[Gen loss: 0.688]
22 	[Dis loss: 0.610] 	[Gen loss: 0.696]


ValueError: setting an array element with a sequence.

In [ ]:
"""Выводим Архитектуру"""
generator_img = tf.keras.utils.plot_model(ccgan.generator, to_file="images/generator.png", show_shapes=False,
                                          show_layer_names=False, dpi=128, show_layer_activations=False)

discriminator_img = tf.keras.utils.plot_model(ccgan.discriminator, to_file="images/discriminator.png", show_shapes=False,
                                              show_layer_names=False, dpi=128, show_layer_activations=False)